In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
print(tf.__version__)

1.14.0


In [34]:
def weight_variable(shape):
    #inital = tf.compat.v1.truncted_normal_initializer(shape, stddev=0.1)
    #inital = tf.compat.v1.random.truncted_normal(shape, stddev=0.1)
    #inital = tf.truncted_normal(shape, stddev=0.1)
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [28]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

定义卷积

In [29]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

定义池化

In [40]:
def max_pooling_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [60]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result
    

搭建框架

In [31]:
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)

In [32]:
x_image = tf.reshape(xs, [-1, 28, 28, 1])

第一个卷积层

In [35]:
W_conv1 = weight_variable([5, 5, 1, 32])

b_conv1 = bias_variable([32])

In [37]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [44]:
h_pool1= max_pooling_2x2(h_conv1)

第二个卷积层

In [45]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

In [46]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

In [47]:
h_pool2 = max_pooling_2x2(h_conv2)

全连接层

In [48]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

In [49]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [50]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [52]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W0225 13:22:55.640947 139750344996672 deprecation.py:506] From <ipython-input-52-c160dee9ec5a>:1: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


输出层

In [53]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [56]:
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

损失函数

In [57]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices = [1]))

损失函数最小化

In [58]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

session定义执行

In [61]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if step % 50 == 0:
        print(compute_accuracy(mnist.test.images, mnist.test.labels))

0.0977
0.7695
0.8706
0.9073
0.9235
0.9337
0.9383
0.948
0.9506
0.954
0.957
0.9595
0.9624
0.9632
0.9667
0.9663
0.9688
0.9682
0.9696
0.9709
